## Turborreactor estacionario

Se tiene un turborreactor volando a 32000ft de altura, a una velocidad de 350KIAS. El turborreactor está compuesto por un difusor de rendimiento adiabático 90%, un compresor de rendimiento 75%, una turbina acoplada mecánicamente con rendimiento del 85%, una cámara de combustión de rendimiento 95% y una tobera isoentrópica. 

En el compresor se consigue una relación de presiones de remanso entre la entrada y la salida de 10:1, y en la cámara de combustión se sufre una caída de preisón de remanso del 8%. Además, se sabe que la tobera está adaptada, de forma que expande la corriente de forma óptima consiguiendo la máxima velocidad de salida.

Dada un área de entrada de $A_1=0.8m^2$ del motor, un dosado de $f=0.0015$ en combustión y un poder calorífico del combustible quemado de $L=43\frac{MJ}{kg}$, se pide:

---
1. Obtener la potencia inyectada a la corriente por el compresor
2. Obtener la temperatura de remanso en cámara de combustión y en turbina
3. Obtener el empuje y el rendimiento motopropulsor
4. Calcular el consumo específico y el impulso ofrecido por el motor

---

Se puede asumir combustión diluida y gas ideal. $R_a=287 \frac{J}{kg K}$ , $\gamma_a=1.4$, $c_p=1000 \frac{J}{kg K}$

## Solución

#### Datos del enunciado:

In [1]:
import numpy as np
## Datos del enunciado
# Datos para modelo de atmósfera
g = 9.81 #m/s^2 aceleración de la gravedad
Tref = 288.15 #K, temperatura ambiente a nivel del mar, referencia
pref = 101325 #Pa, presión ambiente a nivel del mar, referencia
alpha = -0.0065 #ºC/1m gradiente térmico en troposfera
deltaT= 0 #ºC desviación sobre el día estándar, se asume cero.

# Constantes del aire:
cp = 1e3 #J/kg/K
gamma_a = 1.4
Ra = 287 #J/kg/K

## Atmósfera Estándar Internacional:
# Temperatura ISA
def T_isa(deltaT, h):
    T_ = Tref + alpha * h + deltaT
    return T_

# Presión ISA
def p_isa(T0, h):
    return pref * (1 + alpha*h/T0)**(-g/Ra/alpha)


In [2]:
## Datos de vuelo
v0 = 350   # KIAS
h0 = 32000 #ft

v0 = v0 * 1852/3600 #velocidad de vuelo en m/s
h0 = h0 * 0.3048 #altitud de vuelo en m

A1 = 0.8 #m^2

# Presión y temperatura estática local:
T0 = T_isa(deltaT, h0)
p0 = p_isa(Tref, h0)

In [3]:
## Datos del motor:
rend_d    = 0.9   # Rendimiento adiabático del difusor
Pi_c      = 10    # Relación de compresión del compresor
rend_c    = 0.75  # Rendimiento adiabático del compresor
Pi_43     = 0.92  # Caída de presión en cámara de combustión
rend_comb = 0.95  # Rendimiento de la combustión
L         = 42e6  # Poder calorífico del combustible
f         = 0.01  # Dosado de la mezcla
rend_t    = 0.85  # Rendimiento adiabático de la turbina
rend_to   = 1     # Tobera isoentrópica

In [4]:
## Magnitudes de remanso de vuelo:
M0 = v0 /np.sqrt(gamma_a*Ra*T0)
T0t = T0 * (1 + (gamma_a-1)/2 * M0**2)
p0t = p0 * (T0t/T0)**(gamma_a/(gamma_a-1))

print('p0={0:5.0f}Pa; T0={1:3.0f}K, h0={2:5.0f}m; v0={3:4.2f}m/s'.format(p0, T0, h0, v0))
print('p0t={0:5.0f}Pa; T0t={1:3.0f}K, M0={2:1.3f}'.format(p0t, T0t, M0))

p0=27430Pa; T0=225K, h0= 9754m; v0=180.06m/s
p0t=34964Pa; T0t=241K, M0=0.599


In [5]:
## Entrada del motor:
p1t = p0t
T1t = T0t
rho1 = p0/Ra/T0
G1 = rho1 * A1 * v0

print('G1={0:3.1f}kg/s'.format(G1))

G1=61.3kg/s


In [6]:
## Salida del difusor
T2t = T1t
p1 = p0
T1 = T0
p2t = p1 * ((T2t/T1 - 1) * rend_d + 1)**(gamma_a/(gamma_a-1))

print('p2t = {0:6.1f}Pa, T2t = {1}K'.format(p2t, T2t))

p2t = 34151.4Pa, T2t = 240.88898331827758K


In [7]:
## Salida del compresor
p3t = Pi_c*p2t
T3t = T2t * (1+(Pi_c**((gamma_a-1)/gamma_a)-1)/rend_c )

print('p3t={0:6.0f}Pa, T3t={1:4.0f}K'.format(p3t, T3t))

p3t=341514Pa, T3t= 540K


In [8]:
## Salida de la cámara de combustión
p4t = p3t*Pi_43

c=G1*f
G3 = G1
Qcomb=rend_comb * c * L

T4t = T3t + Qcomb/G3/cp

print('p4t={0:6.0f}Pa, T4t={1:4.0f}K'.format(p4t, T4t))

p4t=314193Pa, T4t= 939K


In [9]:
## Salida de la turbina
T5t = T4t - T3t + T2t
p5t = ((T5t/T4t - 1)/rend_t + 1)**(gamma_a/(gamma_a-1)) * p4t

print('p5t={0:6.0f}Pa, T5t={1:4.0f}K'.format(p5t, T5t))

p5t= 60779Pa, T5t= 640K


In [10]:
## Salida del motor:
p9 = p0 #Tobera adaptada
T9 = T5t * ((rend_to * (((p9/p5t)**((gamma_a-1)/gamma_a))-1))+1)
print('p9={0:6.0f}Pa, T9={1:4.0f}K'.format(p9, T9))

p9= 27430Pa, T9= 510K


In [11]:
## Cálculo de empuje, impulso y consumo específico:
v9 = np.sqrt(2*cp*(T5t - T9)) # Velocidad de la corriente a través de la temperatura en 9
E=G1*(v9-v0) # Empuje con combustón diluida y tobera adaptada
Isp = E/G1 # Impulso específico
cE = c/E #consumo Específico

In [12]:
print('v9={0:4.1f}m/s, E={1:5.0f}N, Isp={2:3.1f}m/s, cE={3:0.4e}kg/s/N'.format(v9, E, Isp, cE))

v9=510.1m/s, E=20218N, Isp=330.1m/s, cE=3.0297e-05kg/s/N
